<a href="https://colab.research.google.com/github/Bionerdess/NMA2021/blob/main/NMA2021_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuromatch Academy 2021
### Collaborative Project analyzing data by Steinmetz et. al., 2019
By Anna-Lena Schlenner, Shira Lupkin, Katrina Lee Bartas and Joaquin Herrero

In [1]:
#@title Necesssary imports
from scipy import stats, io
import numpy as np
import os, requests
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from matplotlib import rcParams 
import pandas as pd
import pickle

In [2]:
#@title Data retrieval
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)

In [4]:
#@title Data loading
import numpy as np

alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

In [6]:
#title Retreive relevant data

vis_ses=[]

for s in range(len(alldat)):
  dat=alldat[s]
  if 'VISp' in dat['brain_area']:
    vis_ses.append(s)

alldat = alldat[vis_ses]

variables = [['spks'],['brain_area'],['reaction_time'],['response_time'], 
             ['response'],['contrast_left'],['contrast_right'],['feedback_type'],['bin_size']]
dd = np.array([])

for s in range(alldat.shape[0]):

  newdat = dict()
  dat = alldat[s]

  for key in dat:
    if  np.isin(key,variables):
      newdat[key] = dat[key]
  dd = np.hstack((dd,newdat))

dd.shape

(12,)